# TRABALHO FINAL DE MATEMÁTICA DISCRETA

## PARTICIPANTES

- Leonardo Alexandre da Silva Ferreira - 2º Período - Ciência de Dados - 231708030;
- Matheus Fillype Ferreira de Carvalho - 2º Período - Ciência de Dados - 231708017;
- Sillas Rocha da Costa - 2º Período - Ciência de Dados - 231708014.

## INTRODUÇÃO:

O trabalho tem como objetivo aplicar a teoria de grafos à dinâmica de aeroportos, conforme foi mencionado no relatório inicial que foi enviado. São dois os principais objetivos do projeto: o primeiro é criar um programa que possa desenvolver rotas aéreas complexas usando o algoritmo de Dijkstra; isto é, rotas que incluam mais de uma parada obrigatória. Dessa forma, o programa retornaria todo o 'plano de voo'. O segundo objetivo é identificar aeroportos cruciais, ou seja, aeroportos cuja remoção resultaria no maior número de componentes. Na prática, isso significaria encontrar os aeroportos que, caso deixassem de operar por algum motivo, tornariam inviável o acesso a diversos outros aeroportos.

## EXTRAINDO OS DADOS:

Conseguir os dados para executar o projeto foi, de fato, complicado. Procuramos em sites como o [OpenFlights](https://openflights.org/#), que realmente possuem dados sobre as inúmeras linhas aéreas existentes no planeta. No entanto, o grande problema era extrair todos os dados de uma só vez, uma vez que não existia um mecanismo que possibilitasse o download de todas as informações de uma só vez.

A solução encontrada foi uma base de dados presente no site [Kaggle](https://www.kaggle.com/datasets/open-flights/flight-route-database) com dados sobre linhas aéreas entre mais de 3 mil aeroportos em diversos países em janeiro de 2012. Obviamente, desde então, novos aeroportos foram criados e, consequentemente, novas conexões. Contudo, visto que o objetivo do trabalho é aplicar a teoria vista em aula para problemas da vida real, problemas práticos, acredito que a desatualização da base de dados não seja um problema.

Uma vez com os dados em mãos, teríamos que tratá-los para que pudessem atender às nossas necessidades. Abaixo está o código usado para tratar a base de dados inicial e gerar duas bases de dados derivadas: uma apenas com aeroportos brasileiros e outra com todos. Em relação ao tratamento, um comentário relevante é que utilizamos, além da biblioteca `pandas`, a biblioteca de Python `airportsdata` para traduzir os dados, visto que a base inicial possui apenas o código IATA (International Air Transport Association) dos aeroportos, e precisaríamos de outros dados, como latitude e longitude, para calcular as distâncias entre eles, por exemplo.

In [3]:
import pandas as pd  
import airportsdata
import numpy as np 

#lendo o dataframe
dataframe_bruto = pd.read_csv("database/routes.csv")

#carregando dados da biblioteca
dicionario_dos_aeroportos = airportsdata.load('IATA')

#dados que vamos extrai da biblioteca que não existem no dataframe inicial
lista_de_colunas = ["route_id", "source_airport_icao", "source_airport_name", "source_airport_city", "source_airport_country", "source_airport_elevation",
                     "source_airport_lat", "source_airport_lon", "destination_airport_icao", "destination_airport_name", "destination_airport_city", 
                     "destination_airport_country", "destination_airport_elevation", "destination_airport_lat", "destination_airport_lon"]

lista_de_origens = list(dataframe_bruto[" source airport"])
lista_de_destinos = list(dataframe_bruto[" destination apirport"])
lista_de_dados = list()

#extraindo dados da biblioteca
for cada_origem, cada_destino in zip(lista_de_origens, lista_de_destinos):
    try:
        lista_auxiliar = list()
        lista_auxiliar.append(f"{cada_origem}-{cada_destino}")
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_origem]["icao"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_origem]["name"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_origem]["city"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_origem]["country"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_origem]["elevation"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_origem]["lat"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_origem]["lon"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_destino]["icao"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_destino]["name"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_destino]["city"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_destino]["country"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_destino]["elevation"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_destino]["lat"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_destino]["lon"])
        lista_de_dados.append(lista_auxiliar)
    except:
        lista_de_dados.append(list())

dataframe_filtrado = pd.DataFrame(data=lista_de_dados, columns=lista_de_colunas)
dataframe_bruto["route_id"] = dataframe_filtrado["route_id"]

#criando o dataframe final
dataframe_final = pd.merge(dataframe_bruto, dataframe_filtrado, on='route_id', how='inner')
dataframe_final = dataframe_final.dropna(thresh=2)
dataframe_final = dataframe_final.groupby("route_id", as_index=False).first().reset_index(drop=True)

#filtrando para aeroportos brasileiros
dataframe_final_brasil = dataframe_final[dataframe_final["source_airport_country"] == "BR"].reset_index(drop=True)

""" TRECHO PARA SALVAR AS NOVAS BASES DE DADOS
dataframe_final.to_csv("database\\dataframe_final_mundo.csv")
dataframe_final_brasil.to_csv("database\\dataframe_final_brasil.csv")
"""

print(dataframe_final.shape)

(36800, 24)


## CÁLCULO DE ROTAS:

Como foi exposto no documento enviado com os objetivos do trabalho, um dos nossos objetivos é desenvolver um programa com base no algoritmo de Dijkstra. Esse algoritmo é utilizado para encontrar o menor caminho entre dois vértices em um grafo dirigido, e no nosso caso, será utilizado para encontrar as rotas aéreas mais eficientes com base nas horas de voo.

Obviamente, entre dois vértices, o uso do algoritmo seria relativamente simples. No entanto, a nossa aplicação visa encontrar as rotas mais eficientes em viagens complexas, que envolvem múltiplas paradas. Essas paradas devem estar ordenadas.

No nosso modelo, os aeroportos são representados como vértices, e as rotas aéreas como arestas. Entretanto, em nossa base de dados, não possuímos informações sobre as durações dos voos. Como alternativa, optamos por utilizar latitude e longitude para calcular a distância aproximada entre eles. Posteriormente, calculamos a duração da viagem para um avião comercial médio.

Segue abaixo o código utilizado para calcular a distância entre os aeroportos, levando em consideração a curvatura uniforme da Terra, e a duração aproximada dessas viagens:

In [4]:
import math

def cal_dist(lat_inicial:float, lon_inicial:float, lat_final:float, lon_final:float) -> float:
    """
    """
    try:
        dif_lat = math.radians(float(lat_final) - float(lat_inicial))
        dif_lon = math.radians(float(lon_final) - float(lon_inicial))

        raio_terra_km = 6370

        #funções de haversine para distância esférica
        hav_lat = math.sin(dif_lat/2)**2
        hav_lon = math.sin(dif_lon/2)**2

        cos_1 = math.cos(float(math.radians(lat_inicial)))
        cos_2 = math.cos(float(math.radians(lat_final)))

        dist_real = 2*raio_terra_km*math.asin(math.sqrt(hav_lat + cos_1 * cos_2 * hav_lon))

        return round(dist_real, 3)
    
    except:
        return None


def cal_horas(dist_km:float) -> float:
    """
    """
    try:
        velociade_km_h = 850
        tempo_horas = round(float(dist_km)/velociade_km_h, 3)

        return tempo_horas
    
    except:
        return None

Definidas as funções, vamos agora aplicá-las linha a linha e salvar no nosso dataframe a distância entre os aeroportos em questão, ou seja, o comprimento da rota. Além disso, vamos adicionar ao dataframe a duração estimada da viagem com base na velocidade de um avião comercial médio, como foi mencionado anteriormente. Essa duração será representada no formato decimal, uma vez que esses dados serão utilizados exclusivamente para comparações e operações matemáticas relacionadas à aplicação do algoritmo de Dijkstra.

In [5]:
lista_lat_inicial = list(dataframe_final["source_airport_lat"])
lista_lon_inicial = list(dataframe_final["source_airport_lon"])
lista_lat_final = list(dataframe_final["destination_airport_lat"])
lista_lon_final = list(dataframe_final["destination_airport_lon"])

lista_distancias = list()
lista_tempo_de_viagem = list()
for index in range(0, len(lista_lat_inicial)):

    #extraindo os dados
    lat_inicial = lista_lat_inicial[index]
    lon_inicial = lista_lon_inicial[index]
    lat_final = lista_lat_final[index]
    lon_final = lista_lon_final[index]

    #claculando a distância e o tempo
    dist = cal_dist(lat_inicial, lon_inicial, lat_final, lon_final)
    tempo = cal_horas(dist)

    lista_distancias.append(dist)
    lista_tempo_de_viagem.append(tempo)

#criando as colunas com os novos dados 
dataframe_final["distancia"] = lista_distancias
dataframe_final["tempo_de_viagem"] = lista_tempo_de_viagem

#excluindo linhas, que por algum problema, não foi possível calcular ou a distância, ou o tempo de viagem
dataframe_final = dataframe_final.dropna(thresh=2).reset_index(drop=True)

print(dataframe_final.shape)

(36800, 26)


Após inserir na nossa base de dados as informações necessárias, basta agora criar o programa que recebe o nome de cidades em ordem. Essas cidades devem ter aeroportos, e o programa retorna uma espécie de plano de voo buscando a menor rota entre cada aeroporto. Inicialmente, vamos converter o dataframe para o formato de "grafo", que é um dicionário para cada aeroporto, onde cada chave representa um aeroporto "conectado" a ele e o valor é a distância entre os dois. Abaixo está o código usado para construir o grafo, e no output, um exemplo de como é a estrutura do nosso grafo/dicionário.

In [23]:
grafo_com_pesos = dict()

#criando lista com todos os aeroportos
aeroportos_destino = set(dataframe_final[" destination apirport"])
aeroportos_origem = set(dataframe_final[" source airport"])

#criando os dicionário referente a cada aeroporto
for cada_aeroporto in aeroportos_origem.union(aeroportos_destino):
    grafo_com_pesos[cada_aeroporto] = dict()

#atribuindo nos dicionários de cada aeroporto dos os dicionários adjacentes e suas respectivas distâncias
for cada_aeroporto in aeroportos_destino:
    dataframe_rotas_dest = dataframe_final[dataframe_final[" source airport"] == cada_aeroporto]
    lista_de_destinos = list(dataframe_rotas_dest[" destination apirport"])
    lista_de_distancias = list(dataframe_rotas_dest["distancia"])

    for cada_destino, cada_distancia in zip(lista_de_destinos, lista_de_distancias):
        grafo_com_pesos[cada_aeroporto][cada_destino] = cada_distancia

for cada_aeroporto in aeroportos_origem:
    dataframe_rotas_ori = dataframe_final[dataframe_final[" destination apirport"] == cada_aeroporto]
    lista_de_destinos = list(dataframe_rotas_ori[" source airport"])
    lista_de_distancias = list(dataframe_rotas_ori["distancia"])

    for cada_destino, cada_distancia in zip(lista_de_destinos, lista_de_distancias):
        grafo_com_pesos[cada_aeroporto][cada_destino] = cada_distancia

#deletando eventuais dicionários vazios
grafo_final = grafo_com_pesos.copy()
for cada_chave, cada_aeroporto in grafo_com_pesos.items():
    if len(cada_aeroporto) == 0:
        del grafo_final[cada_chave]

print(grafo_final["CNF"])

{'AEP': 2185.808, 'BEL': 2086.818, 'BPS': 626.662, 'BSB': 590.811, 'CGB': 1359.867, 'CGH': 524.269, 'CKS': 1637.222, 'CPV': 1626.523, 'CWB': 846.069, 'FOR': 1858.123, 'GIG': 361.965, 'GRU': 496.387, 'GYN': 646.628, 'IMP': 1611.648, 'IOS': 748.75, 'IPN': 156.45, 'IZA': 225.836, 'LIS': 7437.936, 'MAO': 2539.109, 'MIA': 6393.73, 'MOC': 324.768, 'POA': 1361.825, 'PTY': 5020.888, 'RAO': 430.656, 'REC': 1607.47, 'SDU': 374.82, 'SLZ': 1894.576, 'SSA': 959.419, 'UBA': 418.366, 'UDI': 453.944, 'VCP': 498.697, 'VDC': 623.805, 'VIX': 391.561, 'NAT': 1792.805, 'PNZ': 1198.08}


In [ ]:
def dijkstra(grafo: dict, origem: str | int, destino: str | int) -> tuple:
    """Recriação do algorítmo de Dijkstra.

    Parameters
    ----------
    grafo : dict
        O grafo com pesos desejado.
    origem : str or int
        o vértice de origem.
    destino : str or int
        o vértice de destino.

    Returns
    -------
    tuple
        uma tupla contendo o menor caminho e a distância do caminho, sendo que cada elemento da lista
        é um vértice do grafo, onde o primeiro é a origem e o último o destino, cada elemento representa
        o próximo vértice do caminho a ser percorrido.
    """
    # Inicializador:
    rotas_pesos = dict()
    for vertice in grafo.keys():
        rotas_pesos[vertice] = np.inf
    rotas_pesos[origem] = 0

    lista_rotas = list()
    visitados = set()
    nao_visitados = set(grafo.keys())
    vertice_atual = origem

    while destino != vertice_atual:

        if len(grafo[vertice_atual]) != 0:
            vizinhos = list(grafo[vertice_atual].keys())
        else:
            break

        print(vertice_atual)

        # Fixando o vértice atual
        visitados.add(vertice_atual)
        if vertice_atual in nao_visitados:
            nao_visitados.remove(vertice_atual)

        # Rotulando vizinhos
        for vizinho in vizinhos:
            peso_vizinho = grafo[vertice_atual][vizinho]
            if rotas_pesos[vizinho] > rotas_pesos[vertice_atual] + peso_vizinho:
                rotas_pesos[vizinho] = rotas_pesos[vertice_atual] + peso_vizinho

            # Criando a lista de rotas para refazer a rota
            if abs(rotas_pesos[vertice_atual] - rotas_pesos[vizinho]) == grafo[vertice_atual][vizinho]:
                menor_rota_atual = [vertice_atual, vizinho, rotas_pesos[vizinho]]
                lista_rotas.append(menor_rota_atual)


        # Escolhendo o novo vértice com o menor peso
        menor_caminho = np.inf

        for cada_vertice in nao_visitados:
            if rotas_pesos[cada_vertice] < menor_caminho:
                menor_caminho = rotas_pesos[cada_vertice]
                novo_vertice_atual = cada_vertice

        vertice_atual = novo_vertice_atual

    # Refazendo o caminho de trás para frente
    lista_rotas.reverse()
    rota_final = [destino]

    while origem != rota_final[-1]:
        vertice_selecionado = rota_final[-1]

        for cada_menor_rota in lista_rotas:
            vertice_1 = cada_menor_rota[0]
            vertice_2 = cada_menor_rota[1]
    
            if vertice_2 == vertice_selecionado:
                if rotas_pesos[vertice_1] == rotas_pesos[vertice_selecionado] - grafo[vertice_1][vertice_2]:
                    rota_final.append(vertice_1)
                    break

    rota_final.reverse()

    return rota_final, rotas_pesos[destino]